# ETL Silver → Gold Layer
## Data Warehouse Dengue 2025 - Star Schema
### Nomenclatura: DICIONARIO_MNEMONICOS.md

**Origem:** `public.dengue_silver` (tabela já transformada pelo ETL Raw→Silver)

**Destino:** Schema `gold` com Star Schema dimensional

In [1]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

# Configuração do banco de dados
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'gis',
    'user': 'postgres',
    'password': 'postgres'
}

BATCH_SIZE = 50_000
print("✅ Setup concluído")

✅ Setup concluído


## 1. Conexão e Verificação Silver Layer

In [2]:
# Conectar ao banco
conn = psycopg2.connect(**DB_CONFIG)
cursor = conn.cursor()

# Verificar dados na Silver (public.dengue_silver)
cursor.execute("""
    SELECT COUNT(*) as total,
           MIN(data_notificacao) as data_min,
           MAX(data_notificacao) as data_max,
           COUNT(DISTINCT uf_sigla) as qtd_ufs
    FROM public.dengue_silver
""")
result = cursor.fetchone()
total_silver = result[0]

print("📊 Silver Layer (public.dengue_silver) - Resumo:")
print(f"   Total registros: {total_silver:,}")
print(f"   Período: {result[1]} a {result[2]}")
print(f"   UFs: {result[3]}")

📊 Silver Layer (public.dengue_silver) - Resumo:
   Total registros: 1,661,634
   Período: 2024-12-29 a 2026-01-05
   UFs: 27


## 2. Carregar Dados Silver em Memória

In [3]:
# Query para carregar todos os dados da Silver Layer
query = """
SELECT 
    id_notificacao,
    uf_sigla,
    data_notificacao,
    data_sintomas,
    data_obito,
    ano_notificacao,
    mes_notificacao,
    semana_epi,
    idade_anos,
    faixa_etaria,
    sexo_desc,
    raca_desc,
    qtd_sintomas,
    qtd_alarmes,
    fl_comorbidade,
    classificacao_desc,
    evolucao_desc,
    fl_confirmado,
    fl_grave,
    fl_obito,
    fl_hospitalizado
FROM public.dengue_silver
"""

print("⏳ Carregando dados Silver para memória...")
df_silver = pd.read_sql(query, conn)
print(f"✅ Carregados {len(df_silver):,} registros")
print(f"   Memória: {df_silver.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

⏳ Carregando dados Silver para memória...
✅ Carregados 1,661,634 registros
   Memória: 920.9 MB


In [4]:
# Verificar amostra dos dados
print("📋 Amostra dos dados carregados:")
df_silver.head(3)

📋 Amostra dos dados carregados:


,id_notificacao,uf_sigla,data_notificacao,data_sintomas,data_obito,ano_notificacao,mes_notificacao,semana_epi,idade_anos,faixa_etaria,...,raca_desc,qtd_sintomas,qtd_alarmes,fl_comorbidade,classificacao_desc,evolucao_desc,fl_confirmado,fl_grave,fl_obito,fl_hospitalizado
0,1514995,SP,2025-03-25,2025-03-21,None,2025,3,12,36.0,20-39 anos,...,Branca,5,0,0,Dengue,Cura,1,0,0,0
1,1514996,SP,2025-03-25,2025-03-23,None,2025,3,12,20.0,20-39 anos,...,Branca,5,0,0,Dengue,Cura,1,0,0,0
2,1514997,SP,2025-03-25,2025-03-21,None,2025,3,12,27.0,20-39 anos,...,Branca,1,0,0,Dengue,Cura,1,0,0,0


## 3. Construção das Dimensões
### 3.1 dim_tmp (Dimensão Tempo)

In [5]:
# Dicionário de dias da semana em português
DIAS_SEMANA = {0: 'Segunda', 1: 'Terça', 2: 'Quarta', 3: 'Quinta', 4: 'Sexta', 5: 'Sábado', 6: 'Domingo'}

# Extrair datas únicas de notificação e sintomas
datas_notif = pd.to_datetime(df_silver['data_notificacao'].dropna().unique())
datas_sint = pd.to_datetime(df_silver['data_sintomas'].dropna().unique())
datas_unicas = pd.Series(list(set(datas_notif) | set(datas_sint))).dropna().unique()
print(f"📅 Datas únicas para dim_tmp: {len(datas_unicas)}")

# Construir dados da dimensão
dim_tempo_data = []
for d in sorted(pd.to_datetime(datas_unicas)):
    data = d.date()
    dim_tempo_data.append({
        'dt_completa': data,
        'nr_ano': d.year,
        'nr_mes': d.month,
        'nr_dia': d.day,
        'nr_trimestre': (d.month - 1) // 3 + 1,
        'nr_semana_epi': d.isocalendar()[1],
        'nr_dia_semana': d.dayofweek + 1,
        'nm_dia': DIAS_SEMANA[d.dayofweek],
        'flag_fim_semana': d.dayofweek >= 5,
        'ds_mes_ano': f"{d.year}-{d.month:02d}",
        'ds_ano_trimestre': f"{d.year}-Q{(d.month - 1) // 3 + 1}"
    })

df_dim_tempo = pd.DataFrame(dim_tempo_data)
print(f"✅ dim_tmp: {len(df_dim_tempo)} registros")

📅 Datas únicas para dim_tmp: 373
✅ dim_tmp: 373 registros


### 3.2 dim_loc (Dimensão Localização)

In [6]:
# Mapeamento de UFs para informações adicionais
UFS_INFO = {
    'AC': ('Acre', 'Norte', 12), 'AL': ('Alagoas', 'Nordeste', 27), 'AP': ('Amapá', 'Norte', 16),
    'AM': ('Amazonas', 'Norte', 13), 'BA': ('Bahia', 'Nordeste', 29), 'CE': ('Ceará', 'Nordeste', 23),
    'DF': ('Distrito Federal', 'Centro-Oeste', 53), 'ES': ('Espírito Santo', 'Sudeste', 32),
    'GO': ('Goiás', 'Centro-Oeste', 52), 'MA': ('Maranhão', 'Nordeste', 21), 'MT': ('Mato Grosso', 'Centro-Oeste', 51),
    'MS': ('Mato Grosso do Sul', 'Centro-Oeste', 50), 'MG': ('Minas Gerais', 'Sudeste', 31),
    'PA': ('Pará', 'Norte', 15), 'PB': ('Paraíba', 'Nordeste', 25), 'PR': ('Paraná', 'Sul', 41),
    'PE': ('Pernambuco', 'Nordeste', 26), 'PI': ('Piauí', 'Nordeste', 22), 'RJ': ('Rio de Janeiro', 'Sudeste', 33),
    'RN': ('Rio Grande do Norte', 'Nordeste', 24), 'RS': ('Rio Grande do Sul', 'Sul', 43),
    'RO': ('Rondônia', 'Norte', 11), 'RR': ('Roraima', 'Norte', 14), 'SC': ('Santa Catarina', 'Sul', 42),
    'SP': ('São Paulo', 'Sudeste', 35), 'SE': ('Sergipe', 'Nordeste', 28), 'TO': ('Tocantins', 'Norte', 17)
}

# Extrair UFs únicas dos dados
ufs_unicas = df_silver['uf_sigla'].dropna().unique()
print(f"🗺️ UFs encontradas: {len(ufs_unicas)}")

# Construir dimensão
dim_loc_data = []
for uf in sorted(ufs_unicas):
    if uf in UFS_INFO:
        info = UFS_INFO[uf]
        dim_loc_data.append({
            'sg_uf': uf,
            'nm_uf': info[0],
            'nm_regiao': info[1],
            'cd_ibge': info[2],
            'nm_capital': 'N/A'
        })

df_dim_loc = pd.DataFrame(dim_loc_data)
print(f"✅ dim_loc: {len(df_dim_loc)} registros")

🗺️ UFs encontradas: 27
✅ dim_loc: 27 registros


### 3.3 dim_pac (Dimensão Paciente)

In [7]:
# Função para determinar faixa etária detalhada
def faixa_etaria_detalhada(idade):
    if pd.isna(idade):
        return 'UNKNOWN'
    if idade < 1:
        return 'Lactente'
    elif idade < 12:
        return 'Criança'
    elif idade < 18:
        return 'Adolescente'
    elif idade < 60:
        return 'Adulto'
    else:
        return 'Idoso'

# Criar colunas auxiliares
df_silver['ds_faixa_etaria'] = df_silver['faixa_etaria'].fillna('Nao informado')
df_silver['ds_sexo'] = df_silver['sexo_desc'].fillna('UNKNOWN')
df_silver['ds_raca'] = df_silver['raca_desc'].fillna('UNKNOWN')
df_silver['ds_faixa_etaria_det'] = df_silver['idade_anos'].apply(faixa_etaria_detalhada)

# Natural key para perfil demográfico
df_silver['nk_demografica'] = (df_silver['ds_faixa_etaria'] + '|' + 
                                df_silver['ds_sexo'] + '|' + 
                                df_silver['ds_raca'])

# Criar dimensão com perfis únicos (subset='nk_demografica' garante unicidade da NK)
df_dim_pac = df_silver[['nk_demografica', 'ds_faixa_etaria', 'ds_sexo', 'ds_raca', 'ds_faixa_etaria_det']].drop_duplicates(subset=['nk_demografica'])
print(f"✅ dim_pac: {len(df_dim_pac)} perfis demográficos únicos")

✅ dim_pac: 123 perfis demográficos únicos


### 3.4 dim_cls (Dimensão Classificação)

In [8]:
# Mapeamento de classificações para informações adicionais
CLASSIF_INFO = {
    'Dengue': ('10', 'Confirmado', 'Leve', 'A90', True),
    'Dengue com Sinais de Alarme': ('11', 'Confirmado', 'Moderado', 'A91', True),
    'Dengue Grave': ('12', 'Confirmado', 'Grave', 'A91', True),
    'Chikungunya': ('13', 'Confirmado', 'Variável', 'A92.0', True),
    'Descartado': ('5', 'Negativo', 'N/A', None, False),
    'Inconclusivo': ('8', 'Indeterminado', 'N/A', None, False),
    'Em investigacao': ('0', 'Em Investigação', 'N/A', None, False)
}

# Extrair classificações únicas dos dados
classifs_unicas = df_silver['classificacao_desc'].dropna().unique()
print(f"📋 Classificações encontradas: {classifs_unicas}")

# Construir dimensão
dim_cls_data = []
for classif in sorted(classifs_unicas):
    if classif in CLASSIF_INFO:
        info = CLASSIF_INFO[classif]
        dim_cls_data.append({
            'cd_classificacao': info[0],
            'ds_classificacao': classif,
            'ds_grupo': info[1],
            'ds_gravidade': info[2],
            'cd_cid': info[3],
            'flag_confirmado': info[4]
        })
    else:
        # Classificação não mapeada
        dim_cls_data.append({
            'cd_classificacao': '99',
            'ds_classificacao': classif,
            'ds_grupo': 'Outros',
            'ds_gravidade': 'N/A',
            'cd_cid': None,
            'flag_confirmado': False
        })

df_dim_cls = pd.DataFrame(dim_cls_data)
print(f"✅ dim_cls: {len(df_dim_cls)} tipos de classificação")

📋 Classificações encontradas: ['Dengue' 'Inconclusivo' 'Dengue com Sinais de Alarme' 'Dengue Grave'
 'Em investigacao']
✅ dim_cls: 5 tipos de classificação


### 3.5 dim_evl (Dimensão Evolução)

In [9]:
# Mapeamento de evoluções para informações adicionais
EVOL_INFO = {
    'Cura': ('1', 'Favorável', False, 'Baixa'),
    'Obito pelo agravo': ('2', 'Óbito', True, 'Crítica'),
    'Obito por outras causas': ('3', 'Óbito', True, 'Crítica'),
    'Obito em investigacao': ('4', 'Óbito', True, 'Crítica'),
    'Ignorado': ('9', 'Indeterminado', False, 'Indeterminada'),
    'Em investigacao': ('0', 'Em Investigação', False, 'Indeterminada')
}

# Extrair evoluções únicas dos dados
evolucoes_unicas = df_silver['evolucao_desc'].dropna().unique()
print(f"📋 Evoluções encontradas: {evolucoes_unicas}")

# Construir dimensão
dim_evl_data = []
for evol in sorted(evolucoes_unicas):
    if evol in EVOL_INFO:
        info = EVOL_INFO[evol]
        dim_evl_data.append({
            'cd_evolucao': info[0],
            'ds_evolucao': evol,
            'ds_tipo_evolucao': info[1],
            'flag_obito': info[2],
            'ds_gravidade_desfecho': info[3]
        })
    else:
        dim_evl_data.append({
            'cd_evolucao': '99',
            'ds_evolucao': evol,
            'ds_tipo_evolucao': 'Outros',
            'flag_obito': False,
            'ds_gravidade_desfecho': 'Indeterminada'
        })

df_dim_evl = pd.DataFrame(dim_evl_data)
print(f"✅ dim_evl: {len(df_dim_evl)} tipos de evolução")

📋 Evoluções encontradas: ['Cura' 'Em investigacao' 'Ignorado' 'Obito por outras causas'
 'Obito pelo agravo' 'Obito em investigacao']
✅ dim_evl: 6 tipos de evolução


### 3.6 dim_snt (Dimensão Sintomas)

In [10]:
# Funções para classificar faixas de sintomas e alarmes
def faixa_sintomas(qtd):
    if pd.isna(qtd) or qtd == 0:
        return 'Nenhum'
    elif qtd <= 2:
        return 'Poucos (1-2)'
    elif qtd <= 5:
        return 'Moderado (3-5)'
    else:
        return 'Muitos (6+)'

def faixa_alarmes(qtd):
    if pd.isna(qtd) or qtd == 0:
        return 'Nenhum'
    elif qtd <= 2:
        return 'Poucos (1-2)'
    else:
        return 'Múltiplos (3+)'

def perfil_clinico(sint, alarm):
    if sint == 0 and alarm == 0:
        return 'Assintomático'
    elif alarm == 0:
        return 'Dengue Clássica'
    elif alarm <= 2:
        return 'Dengue com Alarme'
    else:
        return 'Dengue Grave'

# Criar colunas auxiliares usando qtd_sintomas e qtd_alarmes já existentes
df_silver['ds_faixa_sintomas'] = df_silver['qtd_sintomas'].apply(faixa_sintomas)
df_silver['ds_faixa_alarmes'] = df_silver['qtd_alarmes'].apply(faixa_alarmes)
df_silver['ds_perfil_clinico'] = df_silver.apply(lambda x: perfil_clinico(x['qtd_sintomas'], x['qtd_alarmes']), axis=1)
df_silver['flag_tem_sintomas'] = df_silver['qtd_sintomas'] > 0
df_silver['flag_tem_alarmes'] = df_silver['qtd_alarmes'] > 0

# Natural key
df_silver['nk_sintomas'] = df_silver['ds_faixa_sintomas'] + '|' + df_silver['ds_faixa_alarmes']

# Criar dimensão com combinações únicas
df_dim_snt = df_silver[['nk_sintomas', 'ds_faixa_sintomas', 'ds_faixa_alarmes', 'ds_perfil_clinico', 
                         'flag_tem_sintomas', 'flag_tem_alarmes']].drop_duplicates()
print(f"✅ dim_snt: {len(df_dim_snt)} perfis de sintomas")

✅ dim_snt: 12 perfis de sintomas


## 4. Carga das Dimensões no PostgreSQL

In [11]:
def load_dimension(df, table_name, columns):
    """Carrega uma dimensão no banco de dados"""
    placeholders = ','.join(['%s'] * len(columns))
    sql = f"INSERT INTO {table_name} ({','.join(columns)}) VALUES ({placeholders})"
    
    data = []
    for _, row in df.iterrows():
        record = tuple(row[col] if pd.notna(row[col]) else None for col in columns)
        data.append(record)
    
    cursor.executemany(sql, data)
    conn.commit()
    return len(data)

# Rollback de qualquer transação pendente (em caso de erro anterior)
conn.rollback()

print("⏳ Limpando tabelas existentes...")

# TRUNCATE com RESTART IDENTITY para resetar as sequences
cursor.execute("TRUNCATE TABLE gold.ft_deng RESTART IDENTITY CASCADE")
conn.commit()
cursor.execute("TRUNCATE TABLE gold.dim_tmp RESTART IDENTITY CASCADE")
conn.commit()
cursor.execute("TRUNCATE TABLE gold.dim_loc RESTART IDENTITY CASCADE")
conn.commit()
cursor.execute("TRUNCATE TABLE gold.dim_pac RESTART IDENTITY CASCADE")
conn.commit()
cursor.execute("TRUNCATE TABLE gold.dim_cls RESTART IDENTITY CASCADE")
conn.commit()
cursor.execute("TRUNCATE TABLE gold.dim_evl RESTART IDENTITY CASCADE")
conn.commit()
cursor.execute("TRUNCATE TABLE gold.dim_snt RESTART IDENTITY CASCADE")
conn.commit()
print("✅ Tabelas limpas!")

print("\n⏳ Carregando dimensões no banco de dados...")

# dim_tmp
n = load_dimension(df_dim_tempo, 'gold.dim_tmp', 
    ['dt_completa', 'nr_ano', 'nr_mes', 'nr_dia', 'nr_trimestre', 'nr_semana_epi', 
     'nr_dia_semana', 'nm_dia', 'flag_fim_semana', 'ds_mes_ano', 'ds_ano_trimestre'])
print(f"   dim_tmp: {n} registros")

# dim_loc
n = load_dimension(df_dim_loc, 'gold.dim_loc', 
    ['sg_uf', 'nm_uf', 'nm_regiao', 'cd_ibge', 'nm_capital'])
print(f"   dim_loc: {n} registros")

# dim_pac
n = load_dimension(df_dim_pac, 'gold.dim_pac', 
    ['nk_demografica', 'ds_faixa_etaria', 'ds_sexo', 'ds_raca', 'ds_faixa_etaria_det'])
print(f"   dim_pac: {n} registros")

# dim_cls
n = load_dimension(df_dim_cls, 'gold.dim_cls', 
    ['cd_classificacao', 'ds_classificacao', 'ds_grupo', 'ds_gravidade', 'cd_cid', 'flag_confirmado'])
print(f"   dim_cls: {n} registros")

# dim_evl
n = load_dimension(df_dim_evl, 'gold.dim_evl', 
    ['cd_evolucao', 'ds_evolucao', 'ds_tipo_evolucao', 'flag_obito', 'ds_gravidade_desfecho'])
print(f"   dim_evl: {n} registros")

# dim_snt
n = load_dimension(df_dim_snt, 'gold.dim_snt', 
    ['nk_sintomas', 'ds_faixa_sintomas', 'ds_faixa_alarmes', 'ds_perfil_clinico', 'flag_tem_sintomas', 'flag_tem_alarmes'])
print(f"   dim_snt: {n} registros")

print("\n✅ Todas as dimensões carregadas!")

⏳ Limpando tabelas existentes...
✅ Tabelas limpas!

⏳ Carregando dimensões no banco de dados...
   dim_tmp: 373 registros
   dim_loc: 27 registros
   dim_pac: 123 registros
   dim_cls: 5 registros
   dim_evl: 6 registros
   dim_snt: 12 registros

✅ Todas as dimensões carregadas!


## 5. Criar Lookups para Tabela Fato

In [12]:
print("⏳ Criando lookups das dimensões...")

# Lookup dim_tmp (data -> sk_tmp)
cursor.execute("SELECT sk_tmp, dt_completa FROM gold.dim_tmp")
lk_tmp = {row[1]: row[0] for row in cursor.fetchall()}

# Lookup dim_loc (uf_sigla -> sk_loc)
cursor.execute("SELECT sk_loc, sg_uf FROM gold.dim_loc")
lk_loc = {row[1]: row[0] for row in cursor.fetchall()}

# Lookup dim_pac (nk_demografica -> sk_pac)
cursor.execute("SELECT sk_pac, nk_demografica FROM gold.dim_pac")
lk_pac = {row[1]: row[0] for row in cursor.fetchall()}

# Lookup dim_cls (ds_classificacao -> sk_cls)
cursor.execute("SELECT sk_cls, ds_classificacao FROM gold.dim_cls")
lk_cls = {row[1]: row[0] for row in cursor.fetchall()}

# Lookup dim_evl (ds_evolucao -> sk_evl)
cursor.execute("SELECT sk_evl, ds_evolucao FROM gold.dim_evl")
lk_evl = {row[1]: row[0] for row in cursor.fetchall()}

# Lookup dim_snt (nk_sintomas -> sk_snt)
cursor.execute("SELECT sk_snt, nk_sintomas FROM gold.dim_snt")
lk_snt = {row[1]: row[0] for row in cursor.fetchall()}

print(f"✅ Lookups criados:")
print(f"   dim_tmp: {len(lk_tmp)} datas")
print(f"   dim_loc: {len(lk_loc)} UFs")
print(f"   dim_pac: {len(lk_pac)} perfis")
print(f"   dim_cls: {len(lk_cls)} classificações")
print(f"   dim_evl: {len(lk_evl)} evoluções")
print(f"   dim_snt: {len(lk_snt)} sintomas")

⏳ Criando lookups das dimensões...
✅ Lookups criados:
   dim_tmp: 373 datas
   dim_loc: 27 UFs
   dim_pac: 123 perfis
   dim_cls: 5 classificações
   dim_evl: 6 evoluções
   dim_snt: 12 sintomas


## 6. Preparar e Carregar Tabela Fato

In [13]:
print("⏳ Preparando tabela fato...")

# Preparar DataFrame da fato
df_fato = pd.DataFrame()

# Natural key
df_fato['nk_notificacao'] = df_silver['id_notificacao']

# Foreign keys (usando lookups)
df_fato['fk_tmp'] = df_silver['data_notificacao'].apply(
    lambda x: lk_tmp.get(x, -1) if pd.notna(x) else -1
)

df_fato['fk_loc'] = df_silver['uf_sigla'].apply(
    lambda x: lk_loc.get(x, -1) if pd.notna(x) else -1
)

df_fato['fk_pac'] = df_silver['nk_demografica'].apply(
    lambda x: lk_pac.get(x, -1) if pd.notna(x) else -1
)

df_fato['fk_cls'] = df_silver['classificacao_desc'].apply(
    lambda x: lk_cls.get(x, -1) if pd.notna(x) else -1
)

df_fato['fk_evl'] = df_silver['evolucao_desc'].apply(
    lambda x: lk_evl.get(x, -1) if pd.notna(x) else -1
)

df_fato['fk_snt'] = df_silver['nk_sintomas'].apply(
    lambda x: lk_snt.get(x, -1) if pd.notna(x) else -1
)

# Métricas (já calculadas na Silver Layer)
df_fato['vl_confirmado'] = df_silver['fl_confirmado'].fillna(0).astype(int)
df_fato['vl_grave'] = df_silver['fl_grave'].fillna(0).astype(int)
df_fato['vl_obito'] = df_silver['fl_obito'].fillna(0).astype(int)
df_fato['vl_hospitalizado'] = df_silver['fl_hospitalizado'].fillna(0).astype(int)
df_fato['vl_qtd_sintomas'] = df_silver['qtd_sintomas'].fillna(0).astype(int)
df_fato['vl_qtd_alarmes'] = df_silver['qtd_alarmes'].fillna(0).astype(int)
df_fato['vl_idade_anos'] = df_silver['idade_anos']

# Datas
df_fato['dt_notificacao'] = df_silver['data_notificacao']
df_fato['dt_sintomas'] = df_silver['data_sintomas']

print(f"✅ Tabela fato preparada: {len(df_fato):,} registros")

# Verificar FKs inválidas
for col in ['fk_tmp', 'fk_loc', 'fk_pac', 'fk_cls', 'fk_evl', 'fk_snt']:
    invalidas = (df_fato[col] == -1).sum()
    if invalidas > 0:
        print(f"   ⚠️ {col}: {invalidas:,} FKs inválidas ({invalidas/len(df_fato)*100:.2f}%)")

⏳ Preparando tabela fato...
✅ Tabela fato preparada: 1,661,634 registros


In [ ]:
# Carga da tabela fato em batches
cols = ['nk_notificacao', 'fk_tmp', 'fk_loc', 'fk_pac', 'fk_cls', 'fk_evl', 'fk_snt',
        'vl_confirmado', 'vl_grave', 'vl_obito', 'vl_hospitalizado', 
        'vl_qtd_sintomas', 'vl_qtd_alarmes', 'vl_idade_anos',
        'dt_notificacao', 'dt_sintomas']

sql = f"INSERT INTO gold.ft_deng ({','.join(cols)}) VALUES %s"

total = len(df_fato)
batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
print(f"⏳ Carregando {total:,} registros em {batches} batches...")

for i in range(batches):
    start = i * BATCH_SIZE
    end = min((i + 1) * BATCH_SIZE, total)
    batch = df_fato.iloc[start:end]
    
    data = []
    for _, row in batch.iterrows():
        record = tuple(row[c] if pd.notna(row[c]) else None for c in cols)
        data.append(record)
    
    execute_values(cursor, sql, data)
    conn.commit()
    
    pct = (i + 1) / batches * 100
    print(f"   Batch {i+1}/{batches}: {end:,} registros ({pct:.0f}%)")

print(f"\n✅ Carga concluída: {total:,} registros")

⏳ Carregando 1,661,634 registros em 34 batches...
   Batch 1/34: 50,000 registros (3%)


## 7. Validação Final

In [ ]:
print("🔍 VALIDAÇÃO FINAL")
print("=" * 60)

# Contagem Gold
cursor.execute("SELECT COUNT(*) FROM gold.ft_deng")
total_gold = cursor.fetchone()[0]

print(f"\n📊 VOLUMETRIA:")
print(f"   Silver (public.dengue_silver): {total_silver:,}")
print(f"   Gold (gold.ft_deng): {total_gold:,}")
print(f"   Status: {'✅ OK - Volumes iguais' if total_gold == total_silver else '❌ DIFERENÇA'}")

# Métricas
cursor.execute("""
    SELECT 
        SUM(vl_confirmado) as confirmados,
        SUM(vl_grave) as graves,
        SUM(vl_obito) as obitos,
        SUM(vl_hospitalizado) as hospitalizados
    FROM gold.ft_deng
""")
metricas = cursor.fetchone()

print(f"\n📈 MÉTRICAS EPIDEMIOLÓGICAS:")
print(f"   Casos confirmados: {metricas[0]:,}")
print(f"   Casos graves: {metricas[1]:,}")
print(f"   Óbitos: {metricas[2]:,}")
print(f"   Hospitalizações: {metricas[3]:,}")

if metricas[0] > 0:
    taxa_letalidade = metricas[2] / metricas[0] * 100
    taxa_gravidade = metricas[1] / metricas[0] * 100
    print(f"\n   Taxa de letalidade: {taxa_letalidade:.4f}%")
    print(f"   Taxa de gravidade: {taxa_gravidade:.2f}%")

In [ ]:
# Validação das dimensões
print("\n📋 DIMENSÕES:")
dimensoes = ['dim_tmp', 'dim_loc', 'dim_pac', 'dim_cls', 'dim_evl', 'dim_snt']
for dim in dimensoes:
    cursor.execute(f"SELECT COUNT(*) FROM gold.{dim}")
    count = cursor.fetchone()[0]
    print(f"   {dim}: {count} registros")

In [ ]:
# Top 5 UFs por casos
print("\n🏆 TOP 5 UFs POR CASOS CONFIRMADOS:")
cursor.execute("""
    SELECT l.sg_uf, l.nm_uf, l.nm_regiao,
           SUM(f.vl_confirmado) as casos,
           SUM(f.vl_obito) as obitos
    FROM gold.ft_deng f
    JOIN gold.dim_loc l ON f.fk_loc = l.sk_loc
    GROUP BY l.sg_uf, l.nm_uf, l.nm_regiao
    ORDER BY casos DESC
    LIMIT 5
""")
for row in cursor.fetchall():
    print(f"   {row[0]} ({row[1]}/{row[2]}): {row[3]:,} casos, {row[4]:,} óbitos")

In [ ]:
# Fechar conexão
cursor.close()
conn.close()

print("\n" + "=" * 60)
print("✅ ETL SILVER → GOLD CONCLUÍDO COM SUCESSO!")
print("=" * 60)